In [ ]:
#-----------------------------------------------------------------------------------
# This script solves the quasi-static equation of solid equilibrium "Navier's Eq."
# with the Finite Element Method through Fenicsx open-source simulator. This simulation 
# is two-dimensional plane-strain.Tensile stress is positive.
#-----------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------
# Workflow: Gmsh meshing software --> Fenicsx simulator --> Paraview visualization
#-----------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------
# Wellbore stability analysis
#-----------------------------------------------------------------------------------

![WorkFlow](Workflow.jpeg)
Source: https://computationalmechanics.in/fenics-the-mesh-workflow/

In [1]:
# Mesh from Gmsh
#-----------------------------------------------------------------------------------
from mesh_creation import Gmsh_model # Import meshing function
ms,ms_w,xl,yl,dw = 0.75, 0.01, 10, 10, 0.2032 # mesh size at corners [m], well mesh size [m], x-length [m], y-length [m], wellbore diameter [m]
model,gdim = Gmsh_model(ms,ms_w,xl,yl,dw) # Call meshing function
from mesh_conversion import msh_to_xdmf # Import conversion function
mesh = msh_to_xdmf(model, gdim) # Fenicsx equivalent mesh --> check XDMF folder for mesh

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : [ 90%] Meshing curve 6 (Circle)
Info    : Done meshing 1D (Wall 0.00119475s, CPU 0.001899s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0595126s, CPU 0.059686s)
Info    : 1011 nodes 2027 elements


In [ ]:
# Import several Fenicsx/dolfinx packages
#-----------------------------------------------------------------------------------
from dolfinx import plot, io
from dolfinx.fem import (dirichletbc, Expression, Function, FunctionSpace, 
                         VectorFunctionSpace, TensorFunctionSpace, locate_dofs_topological, Constant)
from dolfinx.fem.petsc import LinearProblem, NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.mesh import locate_entities_boundary, locate_entities, meshtags
from ufl import (TestFunction, TrialFunction, dot, dx, grad, inner, nabla_div, div, Identity, sym, Measure,
                 SpatialCoordinate, lhs, rhs, as_vector, tr, FiniteElement, VectorElement, MixedElement, 
                 split, FacetNormal, TensorElement, as_matrix, as_tensor, atan_2, cos, sin)
from petsc4py.PETSc import ScalarType, Options
import numpy as np
import dolfinx
from mpi4py import MPI

In [ ]:
# Independent elastic constants
#-----------------------------------------------------------------------------------
E = 10e9 # Young's modulus [Pa]
nu = 0.30 # Poisson's ratio [-]

# Dependent elastic constants
#-----------------------------------------------------------------------------------
lambda_ = E*nu/((1+nu)*(1-2*nu)) # Lame parameter [Pa]
mu = E/(2*(1+nu)) # Shear modulus [Pa]
model = "plane_strain"
if model == "plane_stress":
    lambda_ = 2*mu*lambda_/(lambda_+2*mu) # Lame parameter for plane-stress elasticity [Pa]

In [ ]:
# FEM space
#-----------------------------------------------------------------------------------
disp = VectorElement("CG", mesh.ufl_cell(), 2) # Vector piecewise quadratic Lagrange element
scalar = FiniteElement("CG", mesh.ufl_cell(), 2) # Scalar piecewise quadratic Lagrange element
ten = TensorElement("CG", mesh.ufl_cell(), 2) # Tensor piecewise quadratic Lagrange element
V0 = FunctionSpace(mesh,disp) # Vector function space
S = FunctionSpace(mesh, ten) # Tensor function space
C = FunctionSpace(mesh, scalar) # Scalar function space

In [ ]:
# Define inital stress and wellbore pressure
#-----------------------------------------------------------------------------------
Sxx = -10.0e6 # Total maximum horizontal stress [Pa]
Syy = -10.0e6 # Total minimum horizontal stress [Pa]
Pw = -1.0e6 # Wellbore pressure [Pa]

# Remember compressive stress is negative!

In [ ]:
# Stress and strain definition
#-----------------------------------------------------------------------------------
def epsilon(u):
    return sym(grad(u)) # Strain = 0.5*(grad(u) + grad(u).T)
def sigma(u):
    return lambda_*nabla_div(u)*Identity(u.geometric_dimension()) + 2*mu*epsilon(u) # Stress tensor
def epsilon_v(u):
    return nabla_div(u) # Volumetric strain = tr(epsilon)

In [ ]:
# Trial and test "virtual" displacement operators
#-----------------------------------------------------------------------------------
u = TrialFunction(V0)
v = TestFunction(V0)

In [ ]:
# Bilinear form
#-----------------------------------------------------------------------------------
a = inner(sigma(u),epsilon(v))*dx # a*u=L

In [ ]:
# Boundary conditions
#-----------------------------------------------------------------------------------
x = SpatialCoordinate(mesh) # Locate x,y coordinates of elements
n = FacetNormal(mesh) # normal direction to elements
theta = atan_2(x[1],x[0]) # Radial coordinate "theta"
rotate_1 = as_tensor([[cos(theta), sin(theta), 0], # Rotation matrix cartesion --> radial
                      [-sin(theta), cos(theta), 0],
                      [0,0,0]])
rotate_2 = as_tensor([[cos(theta), -sin(theta), 0], # Rotation matrix cartesion --> radial
                      [sin(theta), cos(theta), 0],
                      [0,0,0]])

# Boundry marker: 1=left, 2=base, 3=right, 4=top, 5=well
boundaries = [(1,lambda x: np.isclose(x[0], -xl/2)),
                  (2,lambda x: np.isclose(x[1], -yl/2)),
                  (3,lambda x: np.isclose(x[0], xl/2)),
                  (4,lambda x: np.isclose(x[1], yl/2)),
                  (5,lambda x: np.isclose(np.sqrt(x[0]**2 + x[1]**2),dw/2))] # pick out the left, bottom, right, top, and wellbore boundary DOFs

In [ ]:
# Locate degrees of freedom for all boundaries
#-----------------------------------------------------------------------------------
facet_indices, facet_markers = [], []
fdim = mesh.topology.dim - 1
for (marker, locator) in boundaries:
    facets = locate_entities(mesh, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full(len(facets), marker))
facet_indices = np.array(np.hstack(facet_indices), dtype=np.int32)
facet_markers = np.array(np.hstack(facet_markers), dtype=np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = meshtags(mesh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

ds = Measure("ds", domain=mesh, subdomain_data=facet_tag) # External integration measure

In [ ]:
# Apply boundary conditions
#-----------------------------------------------------------------------------------
class BoundaryCondition():
    def __init__(self, type, marker, values, ind_1):
        self._type = type
        if type == "Dirichlet":
            u_D = values
            facets = np.array(facet_tag.indices[facet_tag.values == marker])
            dofs = locate_dofs_topological(V0.sub(ind_1), fdim, facets)
            self._bc = dirichletbc(u_D, dofs, V0.sub(ind_1))
        elif type == "Neumann":
            self._bc = values*inner(n, v) * ds(marker)
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(type))
    @property
    def bc(self):
        return self._bc

    @property
    def type(self):
        return self._type

#-----------------------------------------------------------------------------------
# Boundry marker: 1=left, 2=base, 3=right, 4=top, 5=well
# Example entry: BoundaryCondition(type, Marker, value, ind_1)
# 1. Type is either "Dirichlet" or "Neumann".
# 2. Marker is either 1,2,3,4 or 5 depending on the boundary.
# 3.ind_1=0 for x-value, ind_2=2 for y_value, or ind_2=3 for z-value.
#-----------------------------------------------------------------------------------

boundary_conditions = [BoundaryCondition("Dirichlet", 1, ScalarType(0), 0),
                           BoundaryCondition("Dirichlet", 2, ScalarType(0), 1),
                           BoundaryCondition("Neumann", 3, ScalarType(Sxx), 0),
                           BoundaryCondition("Neumann", 4, ScalarType(Syy), 1),
                           BoundaryCondition("Neumann", 5, ScalarType(Pw), 0)]

bcs = []
f = Constant(mesh, ScalarType((0, 0, 0)))
L = dot(f, v) * dx
for condition in boundary_conditions:
    if condition.type == "Dirichlet":
        bcs.append(condition.bc)
    else:
        L += condition.bc

In [ ]:
# Transient solution
#-----------------------------------------------------------------------------------
file = io.XDMFFile(mesh.comm, "XDMF/Solution_.xdmf", "w") # Output file --> visualize in Paraview
file.write_mesh(mesh) # Attach mesh to the file

problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"}) # Define the problem
uh = problem.solve() # Solve for displacements
uh.name = "Displacement"

# Output stress and stain --> visualize in Paraview
expr_1 = Expression(sigma(uh), S.element.interpolation_points)
stress = Function(S)
stress.interpolate(expr_1)
stress.name = "Cartesian Stress Tensor"
    
expr_2 = Expression(epsilon(uh), S.element.interpolation_points)
strain = Function(S)
strain.interpolate(expr_2)
strain.name = "Cartesian Strain Tensor"
    
expr_3 = Expression(epsilon_v(uh), C.element.interpolation_points)
strain_v = Function(C)
strain_v.interpolate(expr_3)
strain_v.name = "Volumetric Strain"
    
expr_4 = Expression(rotate_1*sigma(uh)*rotate_2, S.element.interpolation_points)
stress_r = Function(S)
stress_r.interpolate(expr_4)
stress_r.name = "Polar Stress Tensor"
    
expr_5 = Expression(rotate_1*epsilon(uh)*rotate_2, S.element.interpolation_points)
strain_r = Function(S)
strain_r.interpolate(expr_5)
strain_r.name = "Polar Strain Tensor"
    
# Write all of the above to output file
file.write_function(uh)
file.write_function(stress)
file.write_function(strain)
file.write_function(strain_v)
file.write_function(stress_r)
file.write_function(strain_r)
file.close()